# Artificial Neural Network - MNIST data

In [1]:
import pandas as pd
import numpy as np

In [2]:
def sigmoid(x):
    return 1 / (1 + np.e ** -x)

In [3]:
#returns dictionary with weights randomized
def create_network(num_inputs,num_hidden,num_outputs):
    network = {}
    tempnetwork = {}
    #make num_hidden weights
    for i in range(1,num_hidden+1):
        key = "H"+str(i)
        listOfWeights=[]
        #want num_inputs+1 (for bias) weights per hidden node
        for j in range(num_inputs+1):
            listOfWeights.append(np.random.uniform(-0.05,0.05))
        tempnetwork.update({key : listOfWeights})
    network.update( {"hidden" : tempnetwork })
    
    tempnetwork = {}
    for i in range(1,num_outputs+1):
        key = "O"+str(i)
        listOfWeights=[]
        for j in range(num_hidden+1):
            listOfWeights.append(np.random.uniform(-0.05,0.05))
        tempnetwork.update({key : listOfWeights})
    network.update( {"output" : tempnetwork })
    
    return network

In [4]:
#returns two things:
#1) A list of OutputO's (1 for each output node)
#2) A list of outputH's (1 for each hidden node)
def propogate_forward(example,network):
    Hweights = list(network["hidden"].values())
    Oweights = list(network["output"].values())
    
    inputs = example.tolist()
    inputs.insert(0,1)
    #find net H1,2,3, ..., H_num_hidden
    netH = []
    for i in range(0,len(Hweights)):
        tempSeries = (pd.Series(Hweights[i]))*(pd.Series(inputs))
        netH.append(tempSeries.sum())
    #activation function to get outputH1,2,3,...,H_num_hidden
    for i in range(len(netH)):
        netH[i]=sigmoid(netH[i])
    
    #Copy netH into another list to be returned as outputH.
    outputH=netH[:]
    #Add bias node to front of netH list
    netH.insert(0,1)

    #find net O1,2,3, ... , H_num_outputs
    netO=[]
    for i in range(0,len(Oweights)):
        tempSeries = (pd.Series(Oweights[i]))*(pd.Series(netH))
        netO.append(tempSeries.sum())
    #activation function to get outputO1,2,..etc
    for i in range(len(netO)):
        netO[i]=sigmoid(netO[i])

    return netO,outputH

In [5]:
#takes a single digit 0-9 as input, returns list of [0.01,0.01,...] where index of digit
#is replaced by 0.99. For use calculating errors of output layers.
def digitToList(num):
    tK = [0.01] * 10
    tK[num]=0.99
    return tK

In [6]:
#returns a list of Output node error terms
def calc_errors_output(outputOs,target):
    deltaOs = []
    tK=digitToList(target)
    for i in range(len(outputOs)):
        deltaOs.append(outputOs[i]*(1-outputOs[i])*(tK[i]-outputOs[i]))
        
#    Following for testing error:   #
#    error=0
#    for i in range(len(tK)):
#        error+=0.5*((tK[i]-outputOs[i])**2)
#    print("output error calced, "+str(error))

    return deltaOs

In [7]:
#returns a list of Hidden node error terms
def calc_errors_hidden(deltaOs,outputHs,network):
    Oweights = list(network["output"].values())
    deltaHs = []
    #for each hidden node
    for i in range(len(outputHs)):
        sigmaWKH = 0
        #for each output node
        for j in range(len(deltaOs)):
            #sum the weight from output node j to hidden node i multiplied by the corresponding deltaO of output node J
            #we use i+1 because Oweights[j][0] is weight of bias node - we want to skip this
            sigmaWKH += (Oweights[j][i+1] * deltaOs[j])
        #appened value to deltaH according to algorithm
        deltaHs.append(outputHs[i]*(1-outputHs[i])*sigmaWKH)
    return deltaHs

In [8]:
#returns network with updated weights
def update_weights(example, network, learning_rate, deltaOs, deltaHs, outputHs):
    Hweights = list(network["hidden"].values())
    Oweights = list(network["output"].values())
    #insert 1 into first index of each for bias nodes
    outputHs.insert(0,1)
    example = list(example)
    example.insert(0,1)
    
    #iterate through Hidden node weights
    for i in range(len(Hweights)):
        for j in range(len(Hweights[i])):
            #update each weight as per backpropogation algorithm
            list(network["hidden"].values())[i][j]= (Hweights[i][j]) + learning_rate * deltaHs[i] * example[j]
    #iterate through output node weights        
    for i in range(len(Oweights)):
        for j in range(len(Oweights[i])):
            #update each weight as per backpropogation algorithm
            list(network["output"].values())[i][j]= (Oweights[i][j]) + learning_rate * deltaOs[i] * outputHs[j]
    
    return network

In [9]:
#Does entire training algorithm and returns updated network weights as a dictionary
def train_network(examples,targets,network,epoch=75,learning_rate=0.005):
    print("Starting training with Epoch =",epoch,", learning rate =",learning_rate,"and ",len(examples),"training examples...")
    for i in range(epoch):
        for row in range(len(examples)):
            #propogate forward
            outputOs, outputHs = propogate_forward(examples.loc[row],network)
            #calc errors output
            deltaOs=calc_errors_output(outputOs,targets[row])
            #calc errors hidden
            deltaHs=calc_errors_hidden(deltaOs,outputHs,network)
            #update weights
            network = update_weights(examples.loc[row],network,learning_rate,deltaOs,deltaHs,outputHs)         
    return network

In [10]:
#propogates examples forward through network and sets guess to index of max value in output -
#which also happens to be the corresponding digit. Prints accordingly.
def test(examples,targets,network):
    print("Starting testing with",len(examples),"testing instances...")
    numCorrect=0
    numIncorrect=0
    
    #For each testing instance, propogate forward to get most likely prediction
    for row in range(len(examples)):
        result=propogate_forward(examples.loc[row],network)        
        guess = np.argmax(result[0])
        
        if(guess==targets[row]):
            numCorrect+=1
        else:
            numIncorrect+=1
            
    print("********************\n\n")
    print("Number of testing examples:",len(examples))
    print("Number correctly predicted:",numCorrect)
    print("Number incorrectly predicted:",numIncorrect)
    print("Accuracy:",(numCorrect/(numCorrect+numIncorrect))*100,"%")
    print("\n\n********************")

In [11]:
#read csv files into pandas dataframes
train_df_start = pd.read_csv("assets/training60000.csv", header=None)
train_labels_start = pd.read_csv("assets/training60000_labels.csv", header=None)
test_df = pd.read_csv("assets/testing10000.csv", header=None)
test_labels = pd.read_csv("assets/testing10000_labels.csv", header=None)

#use only first X rows of training dataframe
train_df = train_df_start.iloc[:500]
train_labels = train_labels_start.iloc[:500]

#convert labels to series instead of dataframe
train_labels = train_labels.iloc[:,0]
test_labels = test_labels.iloc[:,0]

#create a network with random weights - 784 inputs, 16 hidden nodes, 10 outputs
testing_network = create_network(784,16,10)
#train this network on 5000 training instances
trained_network = train_network(train_df,train_labels,testing_network,50,0.005)
#test updated network on 10000 testing instances
test(test_df,test_labels,trained_network)

Starting training with Epoch = 50 , learning rate = 0.005 and  500 training examples...
Starting testing with 10000 testing instances...
********************


Number of testing examples: 10000
Number correctly predicted: 1796
Number incorrectly predicted: 8204
Accuracy: 17.96 %


********************
